In [1]:
import geopandas as gpd


In [2]:
DEFAULT_CRS = "EPSG:4326"

# lendos shapefiles



In [9]:
hex_iptu = gpd.read_file("data/WGS84_hex_iptu.gpkg")
hex_contruco = gpd.read_file("data/WGS84_hex_construcao_open_buildings_2023.gpkg")
hex_cnefe = gpd.read_file("data/hex_agg_cnef_sp.gpkg")
hex_census = gpd.read_file("data/WGS84_hex_census_2010.gpkg")


In [10]:
hex_census.head()

,id_hex,D1_012_d_sum,D1_017_d_sum,D1_035_d_sum,D1_044_d_sum,geometry
0,8aa8100c3257fff,122.0,122.0,122.0,122.0,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5..."
1,8aa8100d2447fff,97.0,97.0,97.0,96.0,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5..."
2,8aa8100e1637fff,64.0,64.0,64.0,64.0,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5..."
3,8aa8100f0827fff,249.0,248.0,249.0,248.0,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6..."
4,8aa81010ec07fff,0.0,0.0,0.0,0.0,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8..."


In [6]:
def set_default_crs(gdf):
    return gdf.to_crs(DEFAULT_CRS)

In [11]:
hex_iptu = set_default_crs(hex_iptu)
hex_contruco = set_default_crs(hex_contruco)
hex_cnefe = set_default_crs(hex_cnefe)
hex_census = set_default_crs(hex_census)


# Agrupando arquivos

In [12]:

hex_f = hex_iptu.join(hex_contruco[['id_hex', 'area_sum', 'area_mean', 'area_median', 'area_std', 'area_var', 'area_count', 'area_const']].set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_cnefe[['id_hex', 'esp_1', 'esp_sum']].set_index("id_hex"), how='left', on='id_hex')
hex_f = hex_f.join(hex_census[['id_hex', 'D1_012_d_sum', 'D1_017_d_sum', 'D1_035_d_sum', 'D1_044_d_sum']].set_index("id_hex"), how='left', on='id_hex')




In [13]:
hex_f.head()

,id_hex,n_lancamentos,geometry,area_sum,area_mean,area_median,area_std,area_var,area_count,area_const,esp_1,esp_sum,D1_012_d_sum,D1_017_d_sum,D1_035_d_sum,D1_044_d_sum
0,8aa8100c3257fff,NaN,"POLYGON ((-46.63592 -23.53032, -46.63664 -23.5...",9316.314802,147.878013,90.353599,156.286531,24425.479717,63.0,0.517499,114.0,203.0,122.0,122.0,122.0,122.0
1,8aa8100d2447fff,10234.0,"POLYGON ((-46.53650 -23.53972, -46.53722 -23.5...",8329.150087,80.865535,61.867680,70.215211,4930.175810,103.0,0.462490,157.0,167.0,97.0,97.0,97.0,96.0
2,8aa8100e1637fff,64736.0,"POLYGON ((-46.66886 -23.59437, -46.66958 -23.5...",2348.146472,90.313326,71.872491,74.175720,5502.037442,26.0,0.130474,70.0,82.0,64.0,64.0,64.0,64.0
3,8aa8100f0827fff,NaN,"POLYGON ((-46.59115 -23.61353, -46.59187 -23.6...",10516.949009,60.442236,46.407855,54.715864,2993.825762,174.0,0.584218,471.0,521.0,249.0,248.0,249.0,248.0
4,8aa81010ec07fff,8.0,"POLYGON ((-46.65231 -23.83331, -46.65303 -23.8...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,0.0,0.0,0.0,0.0


# Transformações finais
- fill all nan

In [28]:
hex_f["hex_fid"] = hex_f["fid"].astype(int) #geopandas as a bug with fid https://github.com/geopandas/geopandas/issues/1035
hex_f.drop("fid", inplace=True, axis=1)

In [14]:
hex_f.fillna(0, inplace=True)

In [16]:
hex_f.to_file("data/WGS84_hex_agg.gpkg", driver="GPKG")